## Active Learning e Adicionando Features de Texto do Título

Aqui aindas estamos em um passo hibrido. Criando modelo mas ainda nao saimos da parte de preparação dos dados, pq apesar de termos alguns dados anotados, 500 dados é muito pouco para modelarmos esse tipo de tarefa, principalmente considerando que só 14% são dados positivos, da classe positivo, entao queremos ter mais dados pra que a gente possa modela.

Active learning serve pra quando temos orçamento pequeno, então é muito caro fazer anotações em novos exemplos, ou muito pouco tempo para fazer essas anotações, ao inves de sair anotando aleatoriamente os exemplos, vamos fazer as anotações em um determinado numero de exemplos que tenha possibilidade de nos trazer mais ganhos por custo de anotação.

Isso é muito importante em dados médicos, pois precisamos de especialistas para estarem avaliando imagens, casos para podes colocar as anotações, se uma radiografia ou ressonancia, tem a doença ou tumor, nesse caso temos uma capacidade limitada e devemos aproveitar ao maximo o tempo desses profissionais. Então assim, seleciona exemplos que tem maior chance de ajudar o modelo a ter uma perfomance melhor ao invés de fazer aleatoriamente.

Esse notebook é parecido com o anterior, a diferença que vamos adicionar o texto do titulo e começar a usar uma Random Forest.

Carregamos os exemplos anotados, limpeza de data e views exatamente igual.



In [1]:
import pandas as pd
import numpy as np
import re 
import time


import bs4 as bs4 
import json

import tqdm
import glob

pd.set_option("display.max_columns",200)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df = df[df['y'].notnull()]
df.shape

(498, 16)

In [3]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

## 1. Limpeza dos Dados

In [4]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## 2. Limpeza de Views

In [5]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## 3. Features

In [6]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [7]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [8]:
features.head()

,views,views_por_dia
0,28028,61.464912
1,1131,2.960733
2,1816,8.446512
3,1171,10.455357
4,1228,3.336957


Aqui, ao invés de fazer igual no outro notebook, que colocamos a seleção de dados na mesma linha que criamos Xtrain, e Xval, criamos mascaras, séries de valores verdadeiros e falsos que selecionam as linhas de acordo com as condições que passei na criar a variavel. Fizemos assim pra economizar espaço

In [9]:
mask_train = df_limpo['date'] < "2019-04-01"
mask_val = df_limpo['date'] >= "2019-04-01"

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((228, 2), (270, 2), (228,), (270,))

##### AQUI COMEÇA A NOVIDADE

Que é: como a gente vai fazer pra extrair os textos desses videos, ML nao vai entender se simplesmente passar string com os titulos pra ele, então precisamos transformar a string em numeros, algum tipo de numero.

Uma das maneiras mais simples de fazer isso e criar uma matriz com a contagem de palavras.

Então poderia simplesmente, colocar em cada linha um video e crio uma matriz que cada coluna acaba sendo uma palavra e simplesmente coloco quantas vezes a palavra aparece no cruzamento da linha de determinado video com a palavra que aparece no titulo dele.

Existem maneiras mais avançadas de fazer isso. Mario gosta do TF-IDF Vectorizer, é uma formula que vai dar mais peso para palavras que aparecem bastante em um determinado exemplo, mas não tanto no dataset todo que a gente tem. Então palavras  que aparecerem pouco entre todos os videos, mas aparecerem muito em determinado video vao ter valor maior do que palavras muito comuns. Por exemplo, a palavra 'Machine' e 'Learning', devem aparecer muito, então essas palavras vão ter peso menor.

E é simples de usar esse transformador de texto no sckiti learning.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# selecionamos a coluna titulo, dos exemplos de treino e de validação e vamos passar isso para um vetorizador que vai transformar esse texto em uma matriz
title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']


# chamo de 'bow' pq siginifca 'bag of words', bolsa de palavras, nome que se dá em geral a esse tipo de formato que transformamos as palavras em matriz de documento e termos
# para usarmos TfidfVectorizer , criamos uma instância dele que nem nos modelos, só que ao invés de fit e predict, a gente usar o trasnform, pq é uma transformação
# o que ele vai ta armazenando aqui, através do fit ? Quais palavras que ele viu, ou seja, palavras que não viu, nao terá colunas indicando se estão ou não no documento
# ele vai guardar qual a frequencia delas no nosso dataset, pra depois fazer a multiplicação na fórmula do TD-IDF
# e passamos as séries dos titulos, tanto do treino e da validação.
# na validação usamos apenas 'transform' se não vou estar ensinao palavras pro meu vetorizador, que eu nao saberia na vida real pq eu nao teria esses exemplos ainda, na vida real nao sei titulo dos videos que vão entra amanha
# mas o que é 'min_df=2', ele é um numero minimo que uma palavra precisa aparecer nos dados pra que ela se torne uma coluna


title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Quando coloquei 'title_vec = TfidfVectorizer(min_df=1)', ele achou um shape para title_bow_train de (228,742)

Ou seja, ele achou 742 palavras nos dados que aparecem pelo menos umas vez, ja que criou 742 colunas

In [11]:
# colocando min_df = 2, ele achou 193 palavras que aparecem no minimo 2 vezes
# quer dizer tbm que a maior parte das palavras só aparece em um vídeo
# isso pode atrapalhar o modelo, pois a gente tem um monte coluna que por acidente o modelo pode achar que é preditivo mas não é.
# entao mario diz que esse é o primeiro parametro que gosta de tunar quando esta trabalhando com texto
# a gente vai ver outro parâmetro tbm num proximo modelo, mas como naõ estamos na parte da modelagem, entao colocamos min_df=2 so pra dar uma reduzida no numero das colunas
# na hora da tunagem achamos o parametro ideal
title_bow_train.shape

(228, 193)

PERGUNTAR PARA O MARIO ? DA ONDE VIRIAM ESSES DADOS ZERO ?

Por padrão, TF-IDF ele te da uma matriz sparsa, que é uma matriz otimizada dentro do scipy, que é o pacote irmão do numpy.

O QUE É UMA MATRIZ SPARSA ? Ele só vai armazenar valores que forem diferentes de zero. Ele ta dizendo que tem 1277 elementos, se a gente fosse armazenar todos os elementos na memoria, inclusive os zeros, teriamos (228*193)=44004 elementos, então a gente ta usando muito menos memoria sem perder poder de representação.

1 - 1277/(228*193) = 0.9709799

Temos que cerca de 97% da nossa matriz é composta de zeros, ou seja, ela é sparsa tanto do ponto de vista matematico quanto computacional.

#### Resposta do MARIO FILHO A MINHA PERGUNTA 

PERGUNTA: A partir de 08:20, não consegui entender. Matriz Esparsa ele armazena valores diferentes de zero, mas depois diz que 97% da matriz é composta de zeros. Ficou meio abstrato, não conseguir visualizar. Tipo, como é feita a composição da matriz se ela armazena valores diferentes de zero, e pensarmos que uma linha representada por um vídeo, terá vários valores zero pois a maioria das colunas, que são as palavras, não estarão contidas no seu título.


RESPOSTA: Jarbas, 97% da matriz, matematicamente, é de zeros. Mas como não vale a pena armazenar os zeros, Ela armazena (na memória) apenas valores diferentes de zero. Quando ela for consultar um elemento e ele não estiver armazenado, retorna zero. Intuitivamente pode pensar numa matriz que tem a linha cheia de zeros, sendo números diferentes de zero apenas nos elementos correspondentes a colunas de palavras que estão no título do vídeo.

In [13]:
title_bow_train.to_array

AttributeError: to_array not found

Como que a gente faz quando temos dados que são númericos, variaveis numericas mas tambem de texto.

Não tem problema nenhum juntar essas variaveis. 

In [ ]:
Xtrain.head()

No Xtrain temos as duas variaveis que ja tinhamos, e agora temos o title_bow_train e val com a representação de texto.

O que eu preciso fazer(IMPORTANTE) é usar a função 'hstack' do scipy sparse.

O QUE ELAS FAZEM ? O numpy tbm tem essas funções. Elas pegam matrizes, no caso do exemplo: vetores, e hstack junta elas horizontalmente.

hstack - [1 2]     [3 4]   ---> [1 2 3 4] - 1x4

Se eu passar pra vstack, ele coloca um embaixo do outro.

vstack - [1 2]     [3 4]   ---> [1 2]
                                [3 4] - 2x2
                                
Basicamente, a concatenação horizontal e vertical que estamos lidando aqui.

Importante que seja scipy sparse, pq usando do numpy demora muito, pq nao e otimizado para utilizar matriz sparsa.

In [ ]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [ ]:
# Shape após juntar X_train com title_bow_train e Xval com title_bow_val.
# 193 da representação textual mais 2 das features númericas.
# modo padrão de lidar com essa situação.

Xtrain_wtitle.shape, Xval_wtitle.shape

Agora treinando o modelo. Mario sempre usa random forest.

Modelo simples, colocamos 1000 árvores, 'balanced' pra equilibrar as classes.

In [ ]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight='balanced', n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

In [ ]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [ ]:
average_precision_score(yval, p)

In [ ]:
roc_auc_score(yval, p)

Alguns experimentos que Mario fez mudando min_df pra 1 e 2

ap 0.17683995211083103, auc 0.6036474164133738 - mindf=1
ap 0.1918043901336543, auc 0.5848024316109421 - mindf=2


Quando eu aumento o min_df, de 1 pra 2, average de precision aumenta mas auc cai, e o que eu to procurando são parametros e features que quando eu adiciono ao modelo ele melhore as duas métricas, nesse caso uma ta melhorando e outra ta piorando.

auc ta variando mutio pq tenho poucos exemplos positivos, entao nao to confiando tanto no auc pq temos pouca quantidade dados, e com min_df=2 a chance de overfitign é menor.

RELEMBRAR COMO FUNCIONA A CURVA ROC : https://www.youtube.com/watch?v=Y1XAP6omGzo

RELEMBRAR AVERAGE PRECISION SCORE: https://www.youtube.com/watch?v=QdWidmgLwbw

# ACTIVE LEARNING

Vamos dizer que eu só tenho orçamento para fazer anotações em mais 100 exemplos, então eu seleciono:

70 exemplos que o modelo tenha dificuldade
30 aleatoriedade

Pq eu não seleciono 100 exemplos que o modelo tenha dificuldade ? Não é pq o modelo ta com dificuldades em alguns exemplos que esses serão as unicas dificuldades que o modelo terá, pode ser que ele ainda nem encontrou algum outro tipo de erro ou exemplo que tenha um padrão diferente de tudo que a gente tem ate agora nos nossos dados de treino, por isso é bom deixar uma parte dos dados ainda sendo selecionadas aleatoriamente para o active learning.

In [ ]:
# df nao anotado, selecionei todas as linhas que nao tem y preenchido, coloquei dropna tira todoas as linhas que tem todos os campos 'nan'

df_unlabeled = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df_unlabeled = df_unlabeled[df_unlabeled['y'].isnull()].dropna(how='all')
df_unlabeled.shape

In [ ]:
df_unlabeled.head(1)

In [ ]:
df_limpo_u = pd.DataFrame(index=df_unlabeled.index)
df_limpo_u['title'] = df_unlabeled['watch-title']

# AQUI USAMOS AS MESMAS FUNÇÕES DOS ULTIMOS DOIS CÓDIGOS, LIMPEZA DE DADOS, VIEWS E FEATURES

In [ ]:
clean_date = df_unlabeled['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo_u['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

In [ ]:
df_limpo_u.head()

In [ ]:
views = df_unlabeled['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo_u['views'] = views

In [ ]:
features_u = pd.DataFrame(index=df_limpo_u.index)

In [ ]:
features_u['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo_u['date']) / np.timedelta64(1, 'D')
features_u['views'] = df_limpo_u['views']
features_u['views_por_dia'] = features_u['views'] / features_u['tempo_desde_pub']
features_u = features_u.drop(['tempo_desde_pub'], axis=1)

In [ ]:
features_u.head()

Dessa vez eu não vou criar um novo TF-IDF vectorizer, ja tenho um modelo de random forest treinada anteriormente, entao vamos usa-la pra prever e entao conseguir selecionar os exemplos que o modelo está com dificuldade

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_u = df_limpo_u['title']
title_bow_u = title_vec.transform(title_u)

In [ ]:
title_bow_u

In [ ]:
# Fazendo o hstack
Xu_wtitle = hstack([features_u, title_bow_u])
Xu_wtitle

In [ ]:
#agora é a previsão para esses dados que nao estão anotados
pu = mdl.predict_proba(Xu_wtitle)[:, 1]

In [ ]:
# coloquei uma coluna nova 'p' com as novas previsões
df_unlabeled['p'] = pu

In [ ]:
# embora 'p' pareça uma probabilidade, não é exatamente, pois estamos dando pesos diferentes pra classe positiva, mas não rigorosamente a gente pode entender como probabilidade, mas como uma pontuação
df_unlabeled.head(1)

### Como que eu sei que um exemplo é dificil para o modelo ?

Novamente, vamos ver quais estão proximos de 50% de probabilidade, entre 0.45 e 0.55, ele está mais confuso.

Como se ele tivesse jogando uma moeda para definir qual é a classe. DESSA MANEIRA DESCOBRIMOS A DIFICULDADE

In [ ]:
mask_u = (df_unlabeled['p'] >= 0.45) & (df_unlabeled['p'] <= 0.55)
mask_u.sum()

In [ ]:
# aqui vemos que tem só 12 exemplos, vamos dar uma olhada
# se a gente der anotações para esses exemplos, pode ser que ele consiga puxar a fronteira dele de classificação, e assim ter mais certeza para classificar corretamente
df_unlabeled[mask_u]

#### Como eu quero 70 exemplos, mario expeculou com os valores da mask, até ter o numero que queria de exemplos.

In [ ]:
mask_u = (df_unlabeled['p'] >= 0.26) & (df_unlabeled['p'] <= 1.)
mask_u.sum()

In [ ]:
# todas da serie que possuem 'True' estão com as probabilidades entre as definidas acima
mask_u

In [ ]:
dificeis = df_unlabeled[mask_u]

In [ ]:
# como combinado era 70 entre as mais dificeis e outras 30, aleatorias, definimos isso aproximado aqui
# '~', simplesmente e como se fosse um negado, que traz amostras de tudo que não contém em mask_u
# se quiser definir que sempre sejam os mesmos valores, definir random state
aleatorios = df_unlabeled[~mask_u].sample(31, random_state=0)

In [ ]:
pd.concat([dificeis, aleatorios]).to_csv("active_label1.csv")

COM ESSE CSV, VOU CARREGAR NO GOOGLE SHEET E DEFINIR OS QUE EU GOSTO E NÃO GOSTO PARA ALIMENTAR O MODELO.